<a href="https://colab.research.google.com/github/mostafa-ja/Information-Retrieval/blob/main/python_elasticsearch_colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install elasticsearch==7.14.0
!apt install default-jdk > /dev/null

In [2]:
try:
  import os
  import elasticsearch
  from elasticsearch import Elasticsearch
  import numpy as np
  import pandas as pd
  import sys
  import json
  from ast import literal_eval
  from tqdm import tqdm 
  import datetime
  from elasticsearch import helpers
  
except Exception as e:
  print(f"error: {e}")

In [3]:
# Download & extract Elasticsearch 7.0.0

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [4]:
# Creating daemon instance of elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [5]:
# This part is important, since it takes a little amount of time for instance to load
import time
time.sleep(20)

In [6]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

daemon       209      73 99 00:05 ?        00:00:25 /content/elasticsearch-7.0.0/jdk/bin/java -Xms1g -Xmx1g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.io.tmpdir=/tmp/elasticsearch-12175366843580146609 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Djava.locale.providers=COMPAT -Dio.netty.allocator.type=unpooled -Des.path.home=/content/elasticsearch-7.0.0 -Des.path.conf=/content/elasticsearch-7.0.0/config -Des.distribution.flavor=default

In [7]:
# Check if elasticsearch is running
!curl -sX GET "localhost:9200/"

In [11]:
from elasticsearch import Elasticsearch

# Adds the HTTP header 'Authorization: Basic <base64 username:password>'
es = Elasticsearch(
    "http://localhost:9200",
     timeout=200
    #ca_certs="/path/to/http_ca.crt",
  
)

print(es.ping())


True


In [12]:
es.info()


{'name': '021eee497021',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '0llTbQbvQYK6741DxUTxjA',
 'version': {'number': '7.0.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'b7e28a7',
  'build_date': '2019-04-05T22:55:32.697037Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [13]:
import pandas as pd
df = pd.read_csv('QueryResults.csv')

In [14]:
df.head(2)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,...,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,9515207,333744,2018-03-19 07:56:42,CertainPerformance,2022-10-23 05:48:50,NaN,"Austin, TX, USA","<p>Check out my <a href=""https://github.com/Ce...",45265,4555,...,2020-08-24 13:58:33,2020-08-24 13:58:33,NaN,NaN,NaN,1,NaN,NaN,NaN,CC BY-SA 4.0
1,115145,966823,2009-05-31 16:20:08,CommonsWare,2022-10-22 22:47:55,https://commonsware.com,"Pennsylvania, United States","<p>Mark Murphy, founder of CommonsWare, writes...",269088,13183,...,2020-12-02 00:04:04,2020-12-02 00:04:04,NaN,NaN,NaN,12,NaN,NaN,NaN,CC BY-SA 4.0


In [15]:
df['Body'][0]

'<p>You need to put a <code>.</code> after the <code>?</code> to use optional chaining:</p>\n<pre><code>myArray.filter(x =&gt; x.testKey === myTestKey)?.[0]\n</code></pre>\n<p><a href="https://www.typescriptlang.org/play/#code/MYewdgzgLgBAtgTwCoFNoGkUJgXhgcgCMBDAJ3wG4BYAKFElkQEFTTjs8BtAbxijSiYEALgIAzECHwwAvgBoYvfhiyiiZaTIC61OuGgxSaAK4AbWHmat2AOjEBLcylIAKAB64AfDDc3lgrFwcS2QBIQBKAH4bTgAGHVp6CBBTFBtTEABzFyMIMyhwiiA" rel="noreferrer">Playground link</a></p>\n<p>Using just the <code>?</code> alone makes the compiler think you\'re trying to use the conditional operator (and then it throws an error since it doesn\'t see a <code>:</code> later)</p>\n<p>Optional chaining isn\'t just a TypeScript thing - it is a finished proposal <a href="https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Optional_chaining" rel="noreferrer">in plain JavaScript</a> too.</p>\n<p>It can be used with bracket notation like above, but it can also be used with dot notation property access:</p>\

In [16]:
df.columns

Index(['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'ProfileImageUrl', 'EmailHash', 'AccountId', 'Id.1', 'PostTypeId',
       'AcceptedAnswerId', 'ParentId', 'CreationDate.1', 'DeletionDate',
       'Score', 'ViewCount', 'Body', 'OwnerUserId', 'OwnerDisplayName',
       'LastEditorUserId', 'LastEditorDisplayName', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'FavoriteCount', 'ClosedDate', 'CommunityOwnedDate', 'ContentLicense'],
      dtype='object')

In [17]:
# find out the date format
for i,row in df.iterrows():
  print(row)
  break

Id                                                                 9515207
Reputation                                                          333744
CreationDate                                           2018-03-19 07:56:42
DisplayName                                             CertainPerformance
LastAccessDate                                         2022-10-23 05:48:50
WebsiteUrl                                                             NaN
Location                                                   Austin, TX, USA
AboutMe                  <p>Check out my <a href="https://github.com/Ce...
Views                                                                45265
UpVotes                                                               4555
DownVotes                                                            65695
ProfileImageUrl          https://www.gravatar.com/avatar/34932d3e923ffa...
EmailHash                                                              NaN
AccountId                

In [ ]:
es.indices.delete(index="questions")

In [18]:
request_body = {
	    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },

	    'mappings': {
	        "properties": {
            "ID": {"type": "integer"},
            "CreationDate": {"type": "date","format": "yyyy-MM-dd HH:mm:ss"},
            "UpVotes": {"type": "integer"},
            "Score": {"type": "integer"},
            "Body": {"type": "text"},
            "OwnerUserId": {"type": "integer"},
            "Tags": {"type": "keyword"}
      }}
	}

es.indices.create(index="questions",body=request_body)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'questions'}

In [19]:
from elasticsearch.helpers import bulk

bulk_data = []
for i,row in df.iterrows():
    bulk_data.append(
        {
            "_index": "questions",
            "_id": i,
            "_source":{
            "ID" : row['Id'],
            "CreationDate" : row["CreationDate"],
            "UpVotes" : row["UpVotes"],
            "Score" : row["Score"],
            "Body" : row["Body"],
            "OwnerUserId" : row["OwnerUserId"],
            "Tags" : row["Tags"]
        } 
        }
    )
bulk(es, bulk_data)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.7/http/client.py", line 1373, in getresponse
    response.begin()
  File "/usr/lib/python3.7/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.7/http/client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/http_urllib3.py"

ConnectionTimeout: ignored

In [21]:
len(bulk_data)

100

In [35]:
for i,row in df.iterrows():
  doc = {
      "ID" : row['Id'],
      "CreationDate" : row["CreationDate"],
      "UpVotes" : row["UpVotes"],
      "Score" : row["Score"],
      "Body" : row["Body"],
      "OwnerUserId" : row["OwnerUserId"],
      "Tags" : row["Tags"]
  }
  es.index(index="questions",id=i,body=doc)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.7/http/client.py", line 1373, in getresponse
    response.begin()
  File "/usr/lib/python3.7/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.7/http/client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/elasticsearch/connection/http_urllib3.py"

ConnectionTimeout: ignored

In [ ]:
indices = es.indices.get_alias() 
print(indices)